In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
signed_time_key = "Business Time"
order_no_key = "Business Order NO."
bill_title = "Billing Title"
receiver_key = "Driver's name"
receiver_code = "Driver Code"
zone_key = "destination zone"
fee_key = "Excluding tax amount"
post_code_key = "Postal Code"
weight_key = "CW"
verify_remark_key = "Verify Remark"


In [3]:
# signed_time_key = '签收时间\nDelivery Time'
# order_no_key = '运单号\nTracking Number'
# receiver_key = '收派员\nDriver'
# weight_key = '商家上传重量\nVendor Uploaded Weight'
# post_code_key = '邮编\nPostcode'
# fee_key = '计费\nCharges'
# verify_remark_key = "备注"

In [4]:
# signed_time_key = "业务时间/Business Time"
# order_no_key = "业务单号/Business Order NO."
# bill_title = "计费科目/Billing Title"
# receiver_key = "司机名称/Driver's name"
# fee_key = "不含税金额/Excluding tax amount"
# post_code_key = "派件省份/Delivery Province"
# weight_key = "重量/weight"
# verify_remark_key = "审批备注/Verify Remark"
# city_key = "派件城市/Delivery City"

In [5]:
# folder_path = r"C:\Users\alice\Desktop\最新数据\iMile\20250407-20250413-extra"
folder_path = r"D:\Projects\transport\raw_bill\week19"

# folder_path = r"C:\Users\alice\Desktop\202504\0407-0413\SYD"

# 判断folder_path是否是文件夹
if os.path.isdir(folder_path):
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.xlsx') and not f.startswith('~$')]

    raw_sheets = []

    for file_path in file_paths:
        rs = pd.read_excel(file_path, sheet_name="sheet1") # , usecols=[signed_time_key, order_no_key, bill_title, receiver_key, fee_key, post_code_key, weight_key, verify_remark_key]
        raw_sheets.append(rs)

    full_sheet = pd.concat(raw_sheets)
else:
    full_sheet = pd.read_excel(folder_path, sheet_name="sheet1")



c:\Users\Boyu\anaconda3\envs\py311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
full_sheet = full_sheet[full_sheet[bill_title]!='丢件/损失赔偿']

In [7]:
print("本周总单数为", full_sheet[order_no_key].nunique())
# full_sheet[order_no_key].nunique()

本周总单数为 49388


In [8]:
origin_sheet = full_sheet[full_sheet[verify_remark_key]!='adjustBill']
adjust_sheet = full_sheet[full_sheet[verify_remark_key]=='adjustBill']

In [9]:
origin_sheet.shape

(49180, 48)

In [10]:
adjust_sheet.shape

(802, 48)

In [11]:
# find the records where order_no_key that in adjust_sheet but not in origin_sheet
extra_drivers = adjust_sheet[~adjust_sheet[order_no_key].isin(origin_sheet[order_no_key])]

In [12]:
extra_drivers.shape

(208, 48)

In [13]:
# extra_drivers.to_csv("extra_drivers.csv")

In [14]:
origin_sheet = pd.concat([origin_sheet, extra_drivers])

In [15]:
adjust_sheet = adjust_sheet[~adjust_sheet[order_no_key].isin(extra_drivers[order_no_key])]

In [16]:
adjust_sheet[order_no_key].nunique()

594

In [17]:
origin_sheet.shape

(49388, 48)

In [18]:
adjust_sheet.shape

(594, 48)

In [19]:
merge_sheet = pd.merge(origin_sheet, adjust_sheet, how='left', left_on=[order_no_key], right_on=[order_no_key], suffixes=('', '_adjust'))

In [20]:
merge_sheet.fillna(0, inplace=True)

In [21]:
merge_sheet['fee_after_adjust'] = merge_sheet[fee_key] + merge_sheet[fee_key + '_adjust']


In [22]:
merge_sheet.shape

(49388, 96)

In [23]:
vc = merge_sheet['fee_after_adjust'].value_counts()
total = (vc.index * vc).sum()
print(vc)
print("排除丢包前总价", total)

fee_after_adjust
 2.1    26144
 2.2    15471
 2.0     4365
 2.3     1942
 5.0      686
 4.0      418
-0.2      208
 8.0      154
Name: count, dtype: int64
排除丢包前总价 108427.6


In [24]:
merge_sheet['fee_after_adjust'] = merge_sheet['fee_after_adjust'].apply(lambda x: round(x, 2))

In [25]:
fee_key = 'fee_after_adjust'

### 处理撤销订单

In [26]:
revoke_sheet = merge_sheet[(merge_sheet[fee_key]<0) & (merge_sheet[verify_remark_key]!='adjustBill')]


In [27]:
for idx, row in revoke_sheet.iterrows():
    revoke_order_no = row[order_no_key]
    revoke_receiver = row[receiver_key]
    # 寻找mergesheet中order_no_key为revoke_order_no, 并且receiver_key与revoke_receiver相同的行
    revoke_rows = merge_sheet[(merge_sheet[order_no_key] == revoke_order_no) & (merge_sheet[receiver_key] == revoke_receiver)]
    # 将找到结果中fee_key为负数的行去掉，也将fee_key为正数，且绝对值和去掉的负数相同的行去掉等量的行数
    # 拆分正负fee
    negative_fees = revoke_rows[revoke_rows[fee_key] < 0]
    positive_fees = revoke_rows[revoke_rows[fee_key] > 0].copy()

    # 标记已删除的index
    indexes_to_drop = set(negative_fees.index)

    for neg_idx, neg_row in negative_fees.iterrows():
        abs_val = abs(neg_row[fee_key])
        # 从正fee中找匹配的金额
        matching_rows = positive_fees[positive_fees[fee_key] == abs_val]
        if not matching_rows.empty:
            # 删除第一个匹配的正数行
            match_idx = matching_rows.index[0]
            indexes_to_drop.add(match_idx)
            positive_fees.drop(index=match_idx, inplace=True)

    # 从 merge_sheet 中删除这些行
    merge_sheet.drop(index=indexes_to_drop, inplace=True)


In [28]:
if len(merge_sheet[merge_sheet[receiver_key]==0]) > 0:
    print(f"警告！有数据没有派送员信息！数量：{len(merge_sheet[merge_sheet[receiver_key].isnull()])}")

警告！有数据没有派送员信息！数量：0


In [29]:
merge_sheet['post_region'] = merge_sheet[post_code_key].apply(lambda x: str(x)[0])
merge_sheet['post_region'].value_counts()

post_region
2    25868
4    23312
0      208
Name: count, dtype: int64

In [62]:
# name_sheet = pd.read_csv('data/name_sheet2.csv')
city_name = input("请输入城市名称 (例如: SYD): ").upper()
if city_name.upper() == "SYD":
    name_sheet_path = 'data/name_sheet_syd.csv'
    post_prefix = "2"
    state_name = "New South Wales"
elif city_name.upper() == "ADL":
    name_sheet_path = 'data/name_sheet_ade.csv'
    post_prefix = "5"
    state_name = "South Australia"
elif city_name.upper() == "BNE":
    name_sheet_path = 'data/name_sheet_bne.csv'
    post_prefix = "4"
    state_name = "Queensland"
else:
    print(f"未找到城市 {city_name} 对应的name_sheet文件，将使用默认文件")
    name_sheet_path = 'data/name_sheet2.csv'



In [63]:
name_sheet = pd.read_csv(name_sheet_path)
merge_sheet[post_code_key] = merge_sheet[post_code_key].astype(str)

if post_code_key == "Postal Code":
    raw_sheet = merge_sheet[merge_sheet[post_code_key].str.startswith(post_prefix)]
else:
    raw_sheet = merge_sheet[merge_sheet[post_code_key]==state_name]


In [64]:
raw_sheet.shape

(23312, 97)

In [65]:
# 时间格式为2025-04-13 16:05:59，需要提取日期

dates = raw_sheet[signed_time_key].apply(lambda x: str(x)[:10]).unique()
dates = np.sort(dates)
dates = [str(date)[:10] for date in dates]
folder_name = "_".join([dates[0], dates[-1]])
project_path = os.path.join("output", city_name, folder_name)
project_cache_path = os.path.join(project_path, "cache")
os.makedirs(project_path, exist_ok=True)
os.makedirs(project_cache_path, exist_ok=True)

In [66]:
table_columns = [signed_time_key, order_no_key, bill_title, receiver_key, post_code_key, zone_key, weight_key, fee_key]

In [67]:
raw_sheet[table_columns].to_csv(f'{project_path}/raw_content.csv')

### 获取当周汇总数据

In [68]:
fees = raw_sheet[fee_key].unique()
fees.sort()

In [69]:
raw_sheet = raw_sheet[raw_sheet[fee_key] > 0]

In [70]:
summary_sheet = raw_sheet[fee_key].value_counts().sort_index()

In [71]:
# summary_sheet = summary_sheet[summary_sheet.index > 0]

summary_sheet.name = '数量'

new_data = pd.Series([summary_sheet.sum()], index=['Total'], name='数量')  # 新行数据
summary_sheet = pd.concat([summary_sheet, new_data])  # 使用 pd.concat 添加新行
summary_sheet.index.name = '单价'

In [72]:
summary_sheet.head(20)

单价
2.1      23055
5.0        159
8.0         98
Total    23312
Name: 数量, dtype: int64

In [73]:
summary_sheet.to_csv(f'{project_path}/week_summary.csv')

### 获取每位派送员派送详情

In [74]:
per_person_sum = raw_sheet[receiver_key].value_counts()

In [75]:
per_person_sum.name = '总数'

In [76]:
per_person_sum = pd.DataFrame(per_person_sum)
per_person_sum = per_person_sum.reset_index()

In [77]:
import pandas as pd
import re

# 定义函数，提取公司简称、账号 ID 和账号名称
def extract_account_info(account_str):
    match = re.match(r"^([A-Z]+(?:-[A-Z]+)*)-(\w+)-(.+)$", account_str)
    if match:
        company_short = match.group(1)
        account_id = match.group(2)
        account_name = match.group(3)
        return company_short, account_id, account_name
    return None, None, account_str  # 仅有姓名时，填充 None


In [78]:
# per_person_sum['name_id'] = ["-".join(x.split("-")[:2]) for x in per_person_sum.index.tolist()]
per_person_sum[['company_short', 'account_id', 'account_name']] = per_person_sum[receiver_key].apply(lambda x: pd.Series(extract_account_info(x)))

In [79]:
# 找到新增的名字
name_sheet[['company_short', 'account_id', 'account_name']] = name_sheet['account'].apply(lambda x: pd.Series(extract_account_info(x)))
per_person_sum["account_id_name"] = per_person_sum.apply(lambda row: 
    f"{row['account_id']}-{row['account_name']}" if row["account_id"] else row["account_name"], axis=1)
per_person_sum['account_name'] = per_person_sum['account_name'].apply(lambda x: x.strip())

name_sheet["account_id_name"] = name_sheet.apply(lambda row: 
    f"{row['account_id']}-{row['account_name']}" if row["account_id"] else row["account_name"], axis=1)

name_sheet['account_name'] = name_sheet['account_name'].apply(lambda x: x.strip())


dedup_key = 'account_name'

duplicate_mask = name_sheet.duplicated(subset=[dedup_key], keep=False)
if duplicate_mask.any():
    # 获取所有重复的记录
    duplicates = name_sheet[duplicate_mask].copy()
    
    # 按account_id_name分组，对每组进行处理
    for account_id_name, group in duplicates.groupby(dedup_key):
        if len(group) > 1:  # 如果有重复记录
            # 检查是否有BPE记录
            bpe_record = group[group['company_short'] == 'BPE']
            if not bpe_record.empty:
                # 如果有BPE记录，删除其他记录
                name_sheet = name_sheet[~((name_sheet[dedup_key] == account_id_name) & 
                                        (name_sheet['company_short'] != 'BPE'))]
            else:
                # 如果没有BPE记录，保留第一条记录
                name_sheet = name_sheet[~((name_sheet[dedup_key] == account_id_name) & 
                                        (name_sheet.index != group.index[0]))]


new_names_id = set(per_person_sum[dedup_key].tolist()) - set(name_sheet[dedup_key].tolist())
new_names_id = list(new_names_id)



In [80]:
# 添加到name_sheet
if new_names_id:  # only proceed if there are new names
    new_names = []
    for name_id in new_names_id:
        # Get the full name from per_person_sum DataFrame
        new_person_record = per_person_sum[per_person_sum[dedup_key]==name_id]
        company_short, account_id, account_name = extract_account_info(new_person_record[receiver_key].values[0])
        new_names.append({
            'account': new_person_record[receiver_key].values[0],
            'company_short': company_short,
            'account_id': account_id,
            'account_name': account_name
        })
    
    new_name_sheet = pd.DataFrame(new_names)
    name_sheet = pd.concat([name_sheet, new_name_sheet], ignore_index=True)

In [81]:
name_sheet['account'].drop_duplicates().to_csv(name_sheet_path, index=False)

In [82]:
name_sheet.head()

,account,company_short,account_id,account_name,account_id_name
0,BNE-BPE-5500-Dipak BUDHATHOKI,BNE-BPE,5500,Dipak BUDHATHOKI,5500-Dipak BUDHATHOKI
1,BNE-BPE-5501-BILAL MUHAMMAD,BNE-BPE,5501,BILAL MUHAMMAD,5501-BILAL MUHAMMAD
2,BNE-BPE-5502-Bikalpa DAWADI,BNE-BPE,5502,Bikalpa DAWADI,5502-Bikalpa DAWADI
3,BNE-BPE-5503-Hoi Yee POW,BNE-BPE,5503,Hoi Yee POW,5503-Hoi Yee POW
4,BNE-BPE-5504-Chu Feng LEE,BNE-BPE,5504,Chu Feng LEE,5504-Chu Feng LEE


In [83]:
per_person_sum['总数'].sum()

23312

In [84]:
name_join_sheet = pd.merge(name_sheet, per_person_sum, left_on=dedup_key, right_on=dedup_key, how='left', suffixes=('_obs', ''), sort=False)
name_join_sheet.fillna(0, inplace=True)
name_join_sheet['总数'] = name_join_sheet['总数'].astype(int) # 将总数列转换为整数
# name_sheet = name_sheet.to_json(orient='records')
# name_sheet = json.loads(name_sheet)

In [85]:
if name_join_sheet['总数'].sum() != summary_sheet.sum()//2:
    print(f"总数不匹配: {name_join_sheet['总数'].sum()} != {summary_sheet.sum()//2}")
    raise ValueError(f"总数不匹配: {name_join_sheet['总数'].sum()} != {summary_sheet.sum()//2}")
else:
    print(f"总数匹配: {name_join_sheet['总数'].sum()} == {summary_sheet.sum()//2}")


总数匹配: 23312 == 23312


In [86]:
# print(counts.to_json(force_ascii=False))
# signed_time_key = None
# receiver_key = None
# fee_key = None
raw_sheet[signed_time_key] = raw_sheet[signed_time_key].apply(lambda x: str(x)[:10])
counts = raw_sheet.groupby([receiver_key, signed_time_key, fee_key]).size().reset_index(name='记录数')

In [87]:
# 转换为嵌套 JSON
nested_json = (
    counts.reset_index()  # 将 MultiIndex 转为普通列
    .groupby(receiver_key)  # 按收派员名字分组
    .apply(lambda x: x.groupby(signed_time_key)  # 在组内按签收时间分组
           .apply(lambda y: y.set_index(fee_key)['记录数'].to_dict())  # 将计费和对应数量转为字典
           .to_dict())  # 将签收时间分组转为字典
    .to_dict()  # 将收派员编号分组转为字典
)

In [88]:
name_join_sheet['data'] = name_join_sheet[receiver_key].apply(lambda x: nested_json.get(x, {}))

In [89]:
# columns = ["序号", "编号", "ACCOUNT"]
columns = ["序号", "ACCOUNT"]
sub_columns = [""] * len(columns)
# 定义表头
for d in dates:
    columns.append(d)
    columns.extend([""] * (len(fees) - 1))

    sub_columns.extend(fees)

columns.append("Total")
sub_columns.append("")
# 创建多级表头
header = pd.MultiIndex.from_tuples(list(zip(columns, sub_columns)))

In [90]:
full_data = []

for idx, person in name_join_sheet.iterrows():
    data = [idx+1, person['account']]
    for d in dates:
        if d in person['data']:
            for f in fees:
                if f in person['data'][d]:
                    data.append(person['data'][d].get(f, ""))
                else:
                    data.append("")
        else:
            data.extend([""] * len(fees))
    data.append(person['总数'])
    full_data.append(data)

In [91]:
# 添加汇总行
total_row = ["Total", "总计"]
total_by_date_fee = {}  # 用于存储每个日期和费用类型的总和

# 初始化汇总数据结构
for d in dates:
    total_by_date_fee[d] = {f: 0 for f in fees}

# 计算每个日期和费用类型的总和
for person in name_join_sheet.itertuples():
    for d in dates:
        if hasattr(person, 'data') and d in person.data:
            for f in fees:
                if f in person.data[d]:
                    total_by_date_fee[d][f] += person.data[d][f]

# 构建汇总行数据
for d in dates:
    for f in fees:
        total_row.append(total_by_date_fee[d][f])

# 添加总计数量
total_row.append(name_join_sheet['总数'].sum())

# 将汇总行添加到full_data
full_data.append(total_row)

In [92]:
# 创建 DataFrame
df = pd.DataFrame(full_data, columns=header)

In [93]:
from openpyxl import load_workbook

df.to_excel(f"{project_cache_path}/output.xlsx", index=True)
# 加载并移除空行
wb = load_workbook(f"{project_cache_path}/output.xlsx")
ws = wb.active

# 删除空行（通常是第3行）
ws.delete_rows(3)
# ws.delete_cols(0)

# 保存调整后的文件
wb.save(f"{project_path}/per_person_details.xlsx")
print(f"文件已保存至 {project_path}/per_person_details.xlsx")
wb.close()

文件已保存至 output\BNE\2025-05-05_2025-05-11/per_person_details.xlsx
